In [4]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval
import pandas as pd
from hyperopt.pyll import scope
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt

In [9]:
df = pd.read_csv('C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv', sep=',')
df.head()


,Transaction Hash_len,Original_len,signature_len,From_len,To_len,sender_len,paymaster_len,Txn Fee,Gas Used,logIndex,actualGasCost,actualGasUsed,nonce,success,Blockno,DateTime_ts
0,32,964,4,20,20,20,20,0.022033,398741,245,2.190000e+16,397164,0.0,1,17066994,1681740540
1,32,868,4,20,20,20,20,0.004362,87702,231,4.280000e+15,86113,1.0,1,17067000,1681740600
2,32,868,4,20,20,20,20,0.003971,87714,273,3.900000e+15,86125,2.0,1,17067009,1681740720
3,32,1188,4,20,20,20,20,0.008673,161702,429,8.650000e+15,161161,3.0,1,17078992,1681887000
4,32,1188,4,20,20,20,20,0.006445,127502,393,6.420000e+15,126973,4.0,1,17079029,1681887420


In [33]:
# ===== 0) Imports =====
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

from catboost import CatBoostRegressor

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll import scope

# ===== 1) Load data ===== 
csv_path = r"C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv"
df = pd.read_csv(csv_path)

features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
X = df[features].reset_index(drop=True).astype(float)
y = df['Gas Used'].astype(float).reset_index(drop=True)

# (Tuỳ chọn) log-transform target nếu skew mạnh
USE_LOG_TARGET = False
y_trainable = np.log1p(y) if USE_LOG_TARGET else y.copy()

# ===== 2) Search space =====
def base_space():
    return {
        'depth': scope.int(hp.quniform('depth', 4, 10, 1)),
        'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1e-2), np.log(1e2)),
        'iterations': scope.int(hp.quniform('iterations', 400, 2000, 50)),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
        'bootstrap_type': hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli']),
        'subsample': hp.uniform('subsample', 0.6, 1.0),
        'bagging_temperature': hp.uniform('bagging_temperature', 0.0, 1.0),
        'random_strength': hp.uniform('random_strength', 0.0, 5.0),
        'rsm': hp.uniform('rsm', 0.6, 1.0),
        'leaf_estimation_iterations': scope.int(hp.quniform('leaf_estimation_iterations', 1, 20, 1)),
    }

def trust_space(center, delta=0.25):
    def clamp(v, lo, hi): 
        return float(max(lo, min(hi, v)))
    return {
        'depth': scope.int(hp.quniform(
            'depth',
            max(4, int(center['depth']) - 1),
            min(10, int(center['depth']) + 1), 1
        )),
        'l2_leaf_reg': hp.loguniform(
            'l2_leaf_reg',
            np.log(clamp(center['l2_leaf_reg'] * (1 - delta), 1e-2, 1e2)),
            np.log(clamp(center['l2_leaf_reg'] * (1 + delta), 1e-2, 1e2))
        ),
        'iterations': scope.int(hp.quniform(
            'iterations',
            clamp(center['iterations'] - 300, 200, 3000),
            clamp(center['iterations'] + 300, 200, 3000), 50
        )),
        'learning_rate': hp.loguniform(
            'learning_rate',
            np.log(clamp(center['learning_rate'] * (1 - delta), 0.005, 0.3)),
            np.log(clamp(center['learning_rate'] * (1 + delta), 0.005, 0.3))
        ),
        'bootstrap_type': hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli']),
        'subsample': hp.uniform(
            'subsample',
            clamp(center.get('subsample', 0.8) - delta, 0.5, 1.0),
            clamp(center.get('subsample', 0.8) + delta, 0.5, 1.0)
        ),
        'bagging_temperature': hp.uniform(
            'bagging_temperature',
            clamp(center.get('bagging_temperature', 0.5) - delta, 0.0, 1.0),
            clamp(center.get('bagging_temperature', 0.5) + delta, 0.0, 1.0)
        ),
        'random_strength': hp.uniform(
            'random_strength',
            clamp(center.get('random_strength', 1.0) - 2*delta, 0.0, 10.0),
            clamp(center.get('random_strength', 1.0) + 2*delta, 0.0, 10.0)
        ),
        'rsm': hp.uniform(
            'rsm',
            clamp(center.get('rsm', 0.8) - delta, 0.5, 1.0),
            clamp(center.get('rsm', 0.8) + delta, 0.5, 1.0)
        ),
        'leaf_estimation_iterations': scope.int(hp.quniform(
            'leaf_estimation_iterations',
            max(1, int(center.get('leaf_estimation_iterations', 10)) - 5),
            min(32, int(center.get('leaf_estimation_iterations', 10)) + 5), 1
        )),
    }

# ===== 3) Cross-validation objective (scaled per fold cho cả X & y) =====
kf = KFold(n_splits=10, shuffle=True, random_state=42)

def cv_metrics(params):
    p = params.copy()
    p['depth'] = int(p['depth'])
    p['iterations'] = int(p['iterations'])
    p['leaf_estimation_iterations'] = int(p['leaf_estimation_iterations'])
    p['subsample'] = float(p['subsample'])
    p['rsm'] = float(p['rsm'])

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for tr_idx, va_idx in kf.split(X):
        X_tr, X_va = X.iloc[tr_idx].values, X.iloc[va_idx].values
        y_tr, y_va = y_trainable.iloc[tr_idx].values, y_trainable.iloc[va_idx].values

        # scale per fold
        scaler_X = MinMaxScaler()
        X_tr_s = scaler_X.fit_transform(X_tr)
        X_va_s = scaler_X.transform(X_va)

        scaler_y = MinMaxScaler()
        y_tr_s = scaler_y.fit_transform(y_tr.reshape(-1, 1)).ravel()
        y_va_s = scaler_y.transform(y_va.reshape(-1, 1)).ravel()

        kwargs = dict(
            loss_function='RMSE',
            eval_metric='R2',
            iterations=p['iterations'],
            learning_rate=float(p['learning_rate']),
            depth=p['depth'],
            l2_leaf_reg=float(p['l2_leaf_reg']),
            random_strength=float(p['random_strength']),
            rsm=float(p['rsm']),
            leaf_estimation_iterations=p['leaf_estimation_iterations'],
            random_seed=42,
            verbose=False,
            allow_writing_files=False
        )
        if p['bootstrap_type'] == 'Bayesian':
            kwargs.update(bootstrap_type='Bayesian',
                          bagging_temperature=float(p['bagging_temperature']))
        else:
            kwargs.update(bootstrap_type='Bernoulli',
                          subsample=float(p['subsample']))

        model = CatBoostRegressor(**kwargs)
        model.fit(X_tr_s, y_tr_s, eval_set=(X_va_s, y_va_s),
                  use_best_model=True, early_stopping_rounds=200)

        preds_s = model.predict(X_va_s)

        mse  = mean_squared_error(y_va_s, preds_s)
        rmse = float(np.sqrt(mse))
        mae  = float(mean_absolute_error(y_va_s, preds_s))
        r2   = float(r2_score(y_va_s, preds_s))

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    return {
        'rmse': rmse_list,
        'mse' : mse_list,
        'mae' : mae_list,
        'r2'  : r2_list,
        'mean_r2': float(np.mean(r2_list))
    }

def objective(params):
    out = cv_metrics(params)
    if out is None:
        return {'loss': 1e9, 'status': STATUS_OK}
    return {
        'loss': float(1.0 - out['mean_r2']),
        'status': STATUS_OK,
        'metrics': out,
        'params': params
    }

# ===== 4) Bayesian Optimization + Trust Region =====
TOTAL_EVALS = 30   # tăng lên 60/120 nếu muốn
STEP = 30
TOP_K = 10
trials = Trials()
rng = np.random.default_rng(42)

for i in range(0, TOTAL_EVALS, STEP):
    if i == 0:
        space = base_space()
    else:
        past = [t for t in trials.trials if 'loss' in t['result']]
        past = sorted(past, key=lambda t: t['result']['loss'])
        topk = past[: min(TOP_K, len(past))]
        center = {}
        for k in ['depth','l2_leaf_reg','iterations','learning_rate','subsample',
                  'bagging_temperature','random_strength','rsm','leaf_estimation_iterations']:
            vals = [t['result']['params'][k] for t in topk if k in t['result']['params']]
            center[k] = float(np.mean(vals)) if len(vals) else {
                'learning_rate':0.05,'subsample':0.8,'bagging_temperature':0.5,
                'random_strength':1.0,'rsm':0.8,'leaf_estimation_iterations':10
            }.get(k, 8.0)
        space = trust_space(center, delta=0.25)

    fmin(fn=objective, space=space, algo=tpe.suggest,
         max_evals=i + STEP, trials=trials, rstate=rng)

# ===== 5) Best trial =====
best_trial = min(trials.trials, key=lambda t: t['result']['loss'])
best_r2_scaled = 1.0 - best_trial['result']['loss']
best_params = best_trial['result']['params']

print("\n🏆 Best R² (CV mean, scaled y):", best_r2_scaled)
print("🔧 Best params:", best_params)

# ===== 6) Re-evaluate BEST PARAMS per fold =====
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []
p = dict(best_params)
p['depth'] = int(p['depth'])
p['iterations'] = int(p['iterations'])
p['leaf_estimation_iterations'] = int(p['leaf_estimation_iterations'])

for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
    X_tr, X_va = X.iloc[tr_idx].values, X.iloc[va_idx].values
    y_tr, y_va = y_trainable.iloc[tr_idx].values, y_trainable.iloc[va_idx].values

    scaler_X = MinMaxScaler()
    X_tr_s = scaler_X.fit_transform(X_tr)
    X_va_s = scaler_X.transform(X_va)

    scaler_y = MinMaxScaler()
    y_tr_s = scaler_y.fit_transform(y_tr.reshape(-1, 1)).ravel()
    y_va_s = scaler_y.transform(y_va.reshape(-1, 1)).ravel()

    params = dict(
        loss_function='RMSE',
        eval_metric='R2',
        iterations=p['iterations'],
        learning_rate=float(p['learning_rate']),
        depth=p['depth'],
        l2_leaf_reg=float(p['l2_leaf_reg']),
        random_strength=float(p['random_strength']),
        rsm=float(p['rsm']),
        leaf_estimation_iterations=p['leaf_estimation_iterations'],
        random_seed=42,
        verbose=False,
        allow_writing_files=False,
        use_best_model=True
    )
    if p['bootstrap_type'] == 'Bayesian':
        params.update(bootstrap_type='Bayesian',
                      bagging_temperature=float(p['bagging_temperature']))
    else:
        params.update(bootstrap_type='Bernoulli',
                      subsample=float(p['subsample']))

    model = CatBoostRegressor(**params)
    model.fit(X_tr_s, y_tr_s, eval_set=(X_va_s, y_va_s),
              early_stopping_rounds=200)
    preds_s = model.predict(X_va_s)

    mse  = mean_squared_error(y_va_s, preds_s)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_va_s, preds_s)
    r2   = r2_score(y_va_s, preds_s)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    print(f"Fold {fold} - (scaled) "
          f"MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

print("\n=== Kết quả trung bình (SCALED) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")


100%|██████████| 30/30 [2:25:54<00:00, 291.83s/trial, best loss: 0.047170710942521454] 

🏆 Best R² (CV mean, scaled y): 0.9528292890574785
🔧 Best params: {'bagging_temperature': 0.03808810504137822, 'bootstrap_type': 'Bayesian', 'depth': 9, 'iterations': 1150, 'l2_leaf_reg': 3.157088896921137, 'leaf_estimation_iterations': 18, 'learning_rate': 0.04688120366273992, 'random_strength': 2.0533400767421526, 'rsm': 0.683305874834135, 'subsample': 0.908911282734798}
Fold 1 - (scaled) MSE: 0.000015 | MAE: 0.000649 | RMSE: 0.003849 | R²: 0.967753
Fold 2 - (scaled) MSE: 0.000090 | MAE: 0.001217 | RMSE: 0.009490 | R²: 0.838412
Fold 3 - (scaled) MSE: 0.000004 | MAE: 0.000616 | RMSE: 0.001939 | R²: 0.991723
Fold 4 - (scaled) MSE: 0.000012 | MAE: 0.000630 | RMSE: 0.003471 | R²: 0.976417
Fold 5 - (scaled) MSE: 0.000003 | MAE: 0.000618 | RMSE: 0.001829 | R²: 0.993262
Fold 6 - (scaled) MSE: 0.000023 | MAE: 0.000746 | RMSE: 0.004818 | R²: 0.968091
Fold 7 - (scaled) MSE: 0.000008 | MAE: 0.000652 | RMSE: 

In [ ]:
TOTAL_EVALS = 30   # tăng nếu muốn
STEP = 30
TOP_K = 10
trials = Trials()
rng = np.random.default_rng(SEED)

for i in range(0, TOTAL_EVALS, STEP):
    if i == 0:
        space = base_space()
    else:
        past = [t for t in trials.trials if 'loss' in t['result']]
        past = sorted(past, key=lambda t: t['result']['loss'])
        topk = past[: min(TOP_K, len(past))]
        center = {}
        for k_ in ['depth','l2_leaf_reg','iterations','learning_rate','subsample',
                  'bagging_temperature','random_strength','rsm','leaf_estimation_iterations']:
            vals = [t['result']['params'][k_] for t in topk if k_ in t['result']['params']]
            center[k_] = float(np.mean(vals)) if len(vals) else {
                'learning_rate':0.05,'subsample':0.8,'bagging_temperature':0.5,
                'random_strength':1.0,'rsm':0.8,'leaf_estimation_iterations':10
            }.get(k_, 8.0)
        space = trust_space(center, delta=0.25)

    fmin(fn=objective, space=space, algo=tpe.suggest,
         max_evals=i + STEP, trials=trials, rstate=rng)

# ===== 5) Best trial =====
best_trial = min(trials.trials, key=lambda t: t['result']['loss'])
best_r2_scaled = 1.0 - best_trial['result']['loss']
best_params = best_trial['result']['params']

print("\n🏆 Best R² (CV mean, scaled y):", best_r2_scaled)
print("🔧 Best params:", best_params)

# ===== 6) Repeated KFold (epochs × k) với Best Params -> cats & final_mean =====
EPOCHS = 20
K = 10
cats = np.zeros((EPOCHS, 4))  # mỗi hàng: [RMSE, MSE, MAE, R2] (trên thang scaled)

# ép kiểu bắt buộc
p = dict(best_params)
p['depth'] = int(p['depth'])
p['iterations'] = int(p['iterations'])
p['leaf_estimation_iterations'] = int(p['leaf_estimation_iterations'])
p['subsample'] = float(p['subsample'])
p['rsm'] = float(p['rsm'])

for ep in range(EPOCHS):
    kf_rep = KFold(n_splits=K, shuffle=True, random_state=SEED + ep)

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for fold, (tr_idx, va_idx) in enumerate(kf_rep.split(X), start=1):
        X_tr, X_va = X.iloc[tr_idx].values, X.iloc[va_idx].values
        y_tr, y_va = y_trainable.iloc[tr_idx].values, y_trainable.iloc[va_idx].values

        # scale per fold
        sx = MinMaxScaler()
        X_tr_s = sx.fit_transform(X_tr)
        X_va_s = sx.transform(X_va)

        sy = MinMaxScaler()
        y_tr_s = sy.fit_transform(y_tr.reshape(-1, 1)).ravel()
        y_va_s = sy.transform(y_va.reshape(-1, 1)).ravel()

        # build params
        kwargs = dict(
            loss_function='RMSE',
            eval_metric='R2',
            iterations=p['iterations'],
            learning_rate=float(p['learning_rate']),
            depth=p['depth'],
            l2_leaf_reg=float(p['l2_leaf_reg']),
            random_strength=float(p['random_strength']),
            rsm=float(p['rsm']),
            leaf_estimation_iterations=p['leaf_estimation_iterations'],
            random_seed=SEED + ep,
            verbose=False,
            allow_writing_files=False,
            use_best_model=True
            # GPU? -> task_type='GPU', devices='0'
        )
        if p['bootstrap_type'] == 'Bayesian':
            kwargs.update(bootstrap_type='Bayesian',
                          bagging_temperature=float(p['bagging_temperature']))
        else:
            kwargs.update(bootstrap_type='Bernoulli',
                          subsample=float(p['subsample']))

        model = CatBoostRegressor(**kwargs)
        model.fit(X_tr_s, y_tr_s, eval_set=(X_va_s, y_va_s),
                  early_stopping_rounds=200)

        preds_s = model.predict(X_va_s)

        # metrics trên thang MinMax-scaled của y
        mse  = mean_squared_error(y_va_s, preds_s)
        rmse = float(np.sqrt(mse))
        mae  = mean_absolute_error(y_va_s, preds_s)
        r2   = r2_score(y_va_s, preds_s)

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    # lưu mean metrics của epoch này (qua 10 folds)
    cats[ep, :] = [
        np.mean(rmse_list),
        np.mean(mse_list),
        np.mean(mae_list),
        np.mean(r2_list)
    ]
    print(f"Epoch {ep+1:02d}/{EPOCHS} -> RMSE={cats[ep,0]:.6f} | MSE={cats[ep,1]:.6f} | MAE={cats[ep,2]:.6f} | R2={cats[ep,3]:.6f}")

# mean cuối cùng qua tất cả epochs
final_mean = np.mean(cats, axis=0)

print("\nFinal mean on MinMax-scaled y [RMSE, MSE, MAE, R2]:")
print(final_mean)  # 1 dòng mean cuối cùng
print("\nPer-epoch metrics (rows) on MinMax-scaled y [RMSE, MSE, MAE, R2]:")
print(cats)        # bảng (epochs × 4)
    